In [1]:
import pandas as pd

l_test = []
l_test.append(None)
l_test

[None]

In [6]:
import os

# Get current working directory
print("Current directory:", os.getcwd())

# Change to a new directory
new_dir = "/Users/sqwu/property_api/property_api"
os.chdir(new_dir)

# Verify the change
print("New directory:", os.getcwd())

Current directory: /Users/sqwu/property_api/property_api/demo
New directory: /Users/sqwu/property_api/property_api


In [7]:
import pandas as pd
from sqlalchemy import create_engine
from src.database import db

database = db.SessionLocal()
data = database.query(db.FlatsToRent).all()
df = pd.DataFrame([task.__dict__ for task in data])

# Check for duplicates
duplicates = df[df.duplicated(['unique_id'], keep=False)]
print(f"Found {len(duplicates)} potential duplicates")
print(duplicates.head())

Found 0 potential duplicates
Empty DataFrame
Columns: [_sa_instance_state, rent, property_type, base, number_of_bathroom, link, address, unique_id, location, price, number_of_bedroom, description, run_time]
Index: []


In [3]:
data = [
  {
    "address": "Valencia Tower, London, EC1V",
    "location": "Islington",
    "price": 3600,
    "number_of_bedroom": 1,
    "description": "One-bedroom apartment in the highly anticipated 250 City Road development, EC1V. The property comprises an open plan kitchen living area, bedroom with built in storage and stylish bathroom.",
    "run_time": "2025-05-25",
    "property_type": "Flat",
    "unique_id": "1|1|2025-05-25|06:10:55.078496",
    "rent": "£3,600 pcm",
    "base": "pcm",
    "number_of_bathroom": 1,
    "link": "https://www.rightmove.co.uk/properties/134069828#/?channel=RES_LET"
  },
  {
    "address": "Charlotte Terrace, London, N1 0TS",
    "location": "Islington",
    "price": 1750,
    "number_of_bedroom": 1,
    "description": "Hauzoo are delighted to market this delightful newly refurbished 1 bedroom, 1 bathroom ground floor flat with private garden located in Islington.",
    "run_time": "2025-05-25",
    "property_type": "Flat",
    "unique_id": "1|2|2025-05-25|06:10:55.078496",
    "rent": "£1,750 pcm",
    "base": "pcm",
    "number_of_bathroom": 1,
    "link": "https://www.rightmove.co.uk/properties/162418898#/?channel=RES_LET"
  },]

import pandas as pd
df = pd.DataFrame(data)
df.to_json()

'{"address":{"0":"Valencia Tower, London, EC1V","1":"Charlotte Terrace, London, N1 0TS"},"location":{"0":"Islington","1":"Islington"},"price":{"0":3600,"1":1750},"number_of_bedroom":{"0":1,"1":1},"description":{"0":"One-bedroom apartment in the highly anticipated 250 City Road development, EC1V. The property comprises an open plan kitchen living area, bedroom with built in storage and stylish bathroom.","1":"Hauzoo are delighted to market this delightful newly refurbished 1 bedroom, 1 bathroom ground floor flat with private garden located in Islington."},"run_time":{"0":"2025-05-25","1":"2025-05-25"},"property_type":{"0":"Flat","1":"Flat"},"unique_id":{"0":"1|1|2025-05-25|06:10:55.078496","1":"1|2|2025-05-25|06:10:55.078496"},"rent":{"0":"\\u00a33,600 pcm","1":"\\u00a31,750 pcm"},"base":{"0":"pcm","1":"pcm"},"number_of_bathroom":{"0":1,"1":1},"link":{"0":"https:\\/\\/www.rightmove.co.uk\\/properties\\/134069828#\\/?channel=RES_LET","1":"https:\\/\\/www.rightmove.co.uk\\/properties\\/16

In [2]:
len(l_test)

1

In [4]:
import re

price_str = "£2,400 pcm"
match = re.search(r'\d[\d,]*', price_str)

if match:
    price = int(match.group(0).replace(',', ''))
    print(price)  # Output: 2400
else:
    price = None


2400


In [7]:
import time
import datetime

ts = time.time()
print(ts)
ct = datetime.datetime.now()
print("current time:", ct)

1747491545.5240908
current time: 2025-05-17 15:19:05.524587


In [9]:
import pandas as pd

df = pd.DataFrame({"col1":["a","a","b"],
                   "col2":[1,1,2]})

df.drop_duplicates()

,col1,col2
0,a,1
2,b,2


In [13]:

a =  False
not a

True

In [11]:
len(df)

3

In [10]:
ts = time.time()
ct = datetime.datetime.now()
print("current time:", str(ct).replace(" ","|"))

current time: 2025-05-17|15:52:02.520054
